## Siamese

In [1]:
import os
os.chdir('../../')

In [2]:
from src.OCR.utils import *
from src.OCR.siamese_unified.bit_pytorch.models import KNOWN_MODELS
from src.OCR.demo import *
import torch.utils.data as data
from PIL import Image, ImageOps
import pickle
import numpy as np
from torchvision import transforms
import json
from collections import OrderedDict
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"]="1"

- Create croppped logo for sampled 1000 phishing

In [4]:
# data_folder = '/home/l/liny/ruofan/phishpedia/benchmark/Sampled_phish1000/'
# annot_file = '/home/l/liny/ruofan/phishpedia/benchmark/phish1000_coord.txt'
# os.makedirs('/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop', exist_ok=True)

# for brand in os.listdir(data_folder):
#     img = Image.open(data_folder + brand + '/shot.png')

#     ## get ground-truth 
#     with open(annot_file, 'r') as annotation_file:
#         for num, line in enumerate(annotation_file):
#             annotation = line.strip().split(',')
#             site = ','.join(annotation[:-4])
#             if site == brand:
#                 bbox_data_gt = np.array(list(annotation[-4:]))
#                 if len(bbox_data_gt) != 0:
#                     bboxes_gt = bbox_data_gt[:4]
#                     x_min_gt, y_min_gt, x_max_gt, y_max_gt = list(map(float, bboxes_gt))
#                     gt_bbox = [x_min_gt, y_min_gt, x_max_gt, y_max_gt]
#                     break   
# #     print(gt_bbox)
#     cropped = img.crop((x_min_gt, y_min_gt, x_max_gt, y_max_gt))
#     cropped.save(os.path.join('/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop', brand+'.png'))
#     del gt_bbox


In [5]:
# data_folder = '/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000/'
# annot_file = '/home/l/liny/ruofan/phishpedia/benchmark/benign1000_coord.txt'
# os.makedirs('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000_crop', exist_ok=True)

# for brand in os.listdir(data_folder):
#     img = Image.open(data_folder + brand + '/shot.png')

#     ## get ground-truth 
#     with open(annot_file, 'r') as annotation_file:
#         for num, line in enumerate(annotation_file):
#             annotation = line.strip().split(',')
#             site = ','.join(annotation[:-4])
#             if site == brand:
#                 bbox_data_gt = np.array(list(annotation[-4:]))
#                 if len(bbox_data_gt) != 0:
#                     bboxes_gt = bbox_data_gt[:4]
#                     x_min_gt, y_min_gt, x_max_gt, y_max_gt = list(map(float, bboxes_gt))
#                     gt_bbox = [x_min_gt, y_min_gt, x_max_gt, y_max_gt]
#                     break   
# #     print(gt_bbox)
#     cropped = img.crop((x_min_gt, y_min_gt, x_max_gt, y_max_gt))
#     cropped.save(os.path.join('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000_crop', brand+'.png'))
#     del gt_bbox



- Define dataloader

In [3]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, label_dict,  transform=None, grayscale=False):
        
        self.transform = transform
        self.data_root = data_root
        self.grayscale = grayscale

        with open(label_dict, 'rb') as handle:
            self.label_dict = pickle.load(handle)

        self.classes = list(self.label_dict.keys())

        self.n_data = len(os.listdir(self.data_root))

        self.img_paths = []
        self.labels = []

        for data in os.listdir(self.data_root):
            image_path = data
            label = image_path.split('+')[0]
            
            if brand_converter(label) == 'Microsoft':
                self.labels.append(label)
                
            elif brand_converter(label) == 'DHL Airways':
                self.labels.append('DHL')
                
            elif brand_converter(label) == 'DGI French Tax Authority':
                self.labels.append('DGI (French Tax Authority)')
                
            else:
                self.labels.append(brand_converter(label))

            self.img_paths.append(image_path)

    def __getitem__(self, item):

        img_path, label= self.img_paths[item], self.labels[item]
        img_path_full = os.path.join(self.data_root, img_path)
        
        if self.grayscale:
            img = Image.open(img_path_full).convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        # label = np.array(label,dtype='float32')
        label = self.label_dict[label]
        if self.transform is not None:
            img = self.transform(img)

        return img, label, img_path_full

    def __len__(self):
        return self.n_data

- Load data

In [4]:
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

img_transforms = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=mean, std=std),
    ])


valid_set = GetLoader(data_root='/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop/', 
                    label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json',
                     transform=img_transforms)


In [5]:
with open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json', 'rb') as handle:
    label_dict = pickle.load(handle)

In [6]:
# valid_loader = torch.utils.data.DataLoader(
#   valid_set, batch_size=128, shuffle=False, pin_memory=True, drop_last=False)

valid_loader = torch.utils.data.DataLoader(
  valid_set, batch_size=1, shuffle=False, pin_memory=False, drop_last=False)

In [7]:
len(valid_loader)

1000

- Accuracy function

In [8]:
@torch.no_grad()
def compute_acc(dataloader, model, ocr_model, device):
    correct = 0
    total = 0
#     for b, (x, y) in tqdm(enumerate(dataloader)):
    for b, (x, y, img_path) in tqdm(enumerate(dataloader)):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        
        ocr_emb = ocr_main(image_path=img_path[0], model=ocr_model, height=None, width=None)
        ocr_emb = ocr_emb[0]
        ocr_emb = ocr_emb[None, ...]
        ocr_emb = ocr_emb.to(device)

        logits = model(x, ocr_emb)
        pred_cls = torch.argmax(logits, dim=1)

        correct += torch.sum(torch.eq(pred_cls, y)).item()
        total += y.shape[0]
        
        if (b+1) % 100 == 0:
            print('Accuracy: {:.2f}'.format(correct/total))
            
    print('Accuracy: {:.2f}'.format(correct/total))    
    return correct/total

- Load model (original)

In [12]:
import torch
import torch.nn.functional as F
from torch import nn

class QuantizeRelu(nn.Module):
    def __init__(self, step_size = 0.01):
        super().__init__()
        self.step_size = step_size

    def forward(self, x):
        mask = torch.ge(x, 0).bool() # mask for positive values
        quantize = torch.ones_like(x) * self.step_size
        out = torch.mul(torch.floor(torch.div(x, quantize)), self.step_size) # quantize by step_size
        out = torch.mul(out, mask) # zero-out negative values
        out = torch.abs(out) # remove sign
        return out

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=277, zero_head=True)

# Load weights
weights_path = 'src/OCR/output/targetlist_lr0.01/bit.pth.tar'
weights = torch.load(weights_path, map_location='cpu')
weights = weights['model'] if 'model' in weights.keys() else weights
new_state_dict = OrderedDict()
for k, v in weights.items():
    if k.startswith('module'):
        name = k.split('module.')[1]
    else:
        name = k
    new_state_dict[name]=v

model.load_state_dict(new_state_dict)

# replace relu with defenselayer 
# model.body.block1.unit01.relu = QuantizeRelu()
# model.body.block1.unit02.relu = QuantizeRelu()
# model.body.block1.unit03.relu = QuantizeRelu()

# model.body.block2.unit01.relu = QuantizeRelu()
# model.body.block2.unit02.relu = QuantizeRelu()
# model.body.block2.unit03.relu = QuantizeRelu()
# model.body.block2.unit04.relu = QuantizeRelu()

# model.body.block3.unit01.relu = QuantizeRelu()
# model.body.block3.unit02.relu = QuantizeRelu()
# model.body.block3.unit03.relu = QuantizeRelu()
# model.body.block3.unit04.relu = QuantizeRelu()
# model.body.block3.unit05.relu = QuantizeRelu()
# model.body.block3.unit06.relu = QuantizeRelu()

model.body.block4.unit01.relu = QuantizeRelu()
model.body.block4.unit02.relu = QuantizeRelu()
model.body.block4.unit03.relu = QuantizeRelu()

model.to(device)

# load OCR model
ocr_model = ocr_model_config(checkpoint='src/OCR/demo_downgrade.pth.tar')
ocr_model.to(device)


using cuda.


ModelBuilder(
  (encoder): ResNet_ASTER(
    (layer0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): AsterBlock(
        (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=[2, 2], bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=[2, 2], bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): AsterBlock(
        (conv1): Conv2d

In [20]:
compute_acc(valid_loader, model, ocr_model, device)

101it [00:15,  8.62it/s]

Accuracy: 0.99


202it [00:31,  8.75it/s]

Accuracy: 0.98


302it [00:41, 10.53it/s]

...

901it [01:50,  8.54it/s]

Accuracy: 0.98


1000it [02:02,  8.17it/s]

Accuracy: 0.98
Accuracy: 0.98


0.979

## Attack

In [14]:
from src.adv_attack.attack.Attack import *

criterion = nn.CrossEntropyLoss()
check = adversarial_attack(method='fgsm', model=model, dataloader=valid_loader, 
                           device=device, num_classes=277, save_data=True,
                           use_ocr=True, ocr_model=ocr_model)

acc, _ = check.batch_attack()

1it [00:06,  6.50s/it]

0
Test Accuracy = 1.0


2it [00:14,  7.01s/it]

1
Test Accuracy = 1.0


3it [00:22,  7.17s/it]

...

997
Test Accuracy = 0.9789579158316634


999it [1:33:26,  5.64s/it]

998
Test Accuracy = 0.978978978978979


1000it [1:33:32,  5.61s/it]

999
Test Accuracy = 0.979
Test Accuracy = 979 / 1000 = 0.979


In [16]:
acc

0.0

## BPDA

- With defense

In [10]:
from advertorch.bpda import BPDAWrapper
from advertorch.attacks import LinfPGDAttack

In [25]:
# original model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=180, zero_head=True)
model.load_state_dict(torch.load('/home/l/liny/ruofan/phishpedia/siamese/checkpoints/resnetv2_rgb.pth', map_location=device))
model.to(device)

# BPDA approximation of gradients
defense_layer = BPDAWrapper(QuantizeRelu())

# replace relu with defenselayer 
model.body.block1.unit01.relu = defense_layer
model.body.block1.unit02.relu = defense_layer
model.body.block1.unit03.relu = defense_layer

model.body.block2.unit01.relu = defense_layer
model.body.block2.unit02.relu = defense_layer
model.body.block2.unit03.relu = defense_layer
model.body.block2.unit04.relu = defense_layer

model.body.block3.unit01.relu = defense_layer
model.body.block3.unit02.relu = defense_layer
model.body.block3.unit03.relu = defense_layer
model.body.block3.unit04.relu = defense_layer
model.body.block3.unit05.relu = defense_layer
model.body.block3.unit06.relu = defense_layer

model.body.block4.unit01.relu = defense_layer
model.body.block4.unit02.relu = defense_layer
model.body.block4.unit03.relu = defense_layer

bpda_adversary = LinfPGDAttack(
    model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.05,
    nb_iter=100, eps_iter=0.005, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False, use_ocr=True, ocr_model=ocr_model, ocr_fn=ocr_main)

In [27]:
perturb_correct = 0
total = 0

for cln_data, true_label, img_path in tqdm(valid_loader):
    cln_data, true_label = cln_data.to(device), true_label.to(device)
    bpda_adv = bpda_adversary.perturb(cln_data, true_label, img_path[0])
    
    ocr_emb = ocr_main(image_path=img_path[0], model = ocr_model, 
                       height=None, width=None)
    ocr_emb = ocr_emb[0]
    ocr_emb = ocr_emb[None, ...]
    ocr_emb = ocr_emb.to(device)
    
    logits = model(bpda_adv, ocr_emb)
    pred_cls = torch.argmax(logits, dim=1)
    perturb_correct += torch.sum(torch.eq(pred_cls, true_label)).item()
    total += len(true_label)
    
    print(perturb_correct/total)

 12%|█▎        | 1/8 [21:33<2:30:57, 1293.95s/it]

0.890625


 25%|██▌       | 2/8 [44:51<2:12:30, 1325.09s/it]

0.9140625


 38%|███▊      | 3/8 [1:08:51<1:53:17, 1359.44s/it]

...

0.921875


 88%|████████▊ | 7/8 [2:43:00<23:17, 1397.87s/it]

0.9274553571428571


100%|██████████| 8/8 [2:58:41<00:00, 1340.19s/it]

0.9274924471299094


In [28]:
perturb_correct/total


0.9274924471299094